In [1]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat

In [2]:
import warnings
warnings.simplefilter("ignore")

In [7]:
import pyod
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

In [8]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [43]:
mat_file_list=['arr.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']

In [44]:
mat_file_list

['arr.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

In [45]:
data=loadmat(r"C:\Users\mansi\Documents\LetsUpgrade\Project_1\cardio.mat")

In [46]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [47]:

len(data)


5

In [48]:


df_columns=['Data','#Sample','#Dimensions','Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS','ABOD','IFOREST','FEATUREBAGGING']

In [49]:
##Precision Time and Roc evolution tables creat
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


In [ ]:
from time import time
random_state = np.random.RandomState(42)
for mat_file in mat_file_list:
    print("\n... Processing",mat_file,'.....')
    mat = loadmat(mat_file)
    
    X = mat['X']
    y = mat['y'].ravel()
    
    outliers_fraction = np.count_nonzero(y) / len(y)
    
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = random_state)
    
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    
    classifiers = {'Angle-based Outlier Detector (ABOD)' : ABOD(contamination=outliers_fraction), 
                  'Cluster-based Local Outlier Factor': CBLOF(contamination=outliers_fraction,check_estimator=False ,random_state=random_state),
                  'Feature Bagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
                  'Historgram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
                  'Isolation Forest' : IForest(contamination=outliers_fraction, random_state=random_state),
                  'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                  'Local Outlier Factor (LOF)' : LOF(contamination=outliers_fraction),
                  'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction, random_state=random_state),
                  'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                  'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction, random_state=random_state)}
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1-t0, 4)
        time_list.append(duration)
        
        roc = round(roc_auc_score(y_test, test_scores), 4)
        prn = round(precision_n_scores(y_test, test_scores), 4)
        
        print(f"{clf_name} ROC: {roc}, precision @ rank n: {prn}, execution time: {duration}s' ")
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)
    

In [ ]:
roc_df

In [ ]:
time_df